# 📡 TelecomX — Análisis de Evasión de Clientes (Churn)

**Objetivo:** Identificar los factores que explican la alta tasa de evasión de clientes en Telecom X, limpiar y transformar los datos para entregarlos listos al equipo de ciencia de datos.

**Herramientas:** Python · Pandas · Matplotlib · Seaborn

# 📌 Extracción

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests

print('✅ Librerías importadas correctamente')

In [ ]:
# Extracción de datos desde la API (JSON)
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json'
response = requests.get(url)
data = response.json()

print(f'✅ Datos extraídos: {len(data)} registros')
print(f'Ejemplo de registro:')
print(json.dumps(data[0], indent=2))

In [ ]:
# Normalizar el JSON anidado a un DataFrame plano
df = pd.json_normalize(data)

print(f'Shape del DataFrame: {df.shape}')
df.head(3)

In [ ]:
# Vista general de los datos
print('📋 Tipos de datos:')
print(df.dtypes)
print()
print('📋 Valores nulos por columna:')
print(df.isnull().sum())

# 🔧 Transformación

In [ ]:
# ── 1. Renombrar columnas para mayor claridad ──
df.columns = [
    'customerID', 'Churn',
    'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
    'PhoneService', 'MultipleLines',
    'InternetService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
    'Contract', 'PaperlessBilling', 'PaymentMethod',
    'MonthlyCharges', 'TotalCharges'
]

print('✅ Columnas renombradas')
print(df.columns.tolist())

In [ ]:
# ── 2. Corregir tipos de datos ──

# TotalCharges: contiene espacios vacíos, convertir a numérico
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# SeniorCitizen: de 0/1 a Yes/No para consistencia
df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

print('✅ Tipos de datos corregidos')
print(df[['TotalCharges', 'SeniorCitizen']].dtypes)

In [ ]:
# ── 3. Tratar valores nulos ──
print(f'Nulos en TotalCharges: {df["TotalCharges"].isna().sum()}')

# Clientes con tenure=0 tienen TotalCharges nulo — se imputa con 0
df['TotalCharges'] = df['TotalCharges'].fillna(0)

print(f'Nulos tras imputación: {df["TotalCharges"].isna().sum()}')

In [ ]:
# ── 4. Tratar columna Churn (espacios vacíos) ──
print('Valores únicos en Churn:', df['Churn'].unique())

# Eliminar filas con Churn vacío
df = df[df['Churn'].isin(['Yes', 'No'])].copy()
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

print(f'✅ Churn limpio. Registros restantes: {len(df)}')
print(df['Churn'].value_counts())

In [ ]:
# ── 5. Eliminar duplicados ──
print(f'Duplicados encontrados: {df.duplicated().sum()}')
df = df.drop_duplicates()
print(f'✅ Shape final del DataFrame: {df.shape}')

In [ ]:
# ── 6. Crear columna de gasto diario (feature engineering) ──
df['DailyCharges'] = (df['MonthlyCharges'] / 30).round(2)

print('✅ Columna DailyCharges creada')
df[['MonthlyCharges', 'TotalCharges', 'DailyCharges']].describe()

In [ ]:
# Resumen del DataFrame limpio
print('📋 DataFrame listo para análisis:')
df.info()
df.head()

# 📊 Carga y análisis

In [ ]:
# Configuración visual
sns.set_theme(style='whitegrid', palette='Set2')
plt.rcParams['figure.figsize'] = (10, 5)
CHURN_PALETTE = {0: '#2ecc71', 1: '#e74c3c'}
print('✅ Configuración visual lista')

In [ ]:
# ── Gráfico 1: Distribución general del Churn ──
churn_counts = df['Churn'].value_counts()
labels = ['No Churn', 'Churn']
colors = ['#2ecc71', '#e74c3c']

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pastel
axes[0].pie(churn_counts, labels=labels, autopct='%1.1f%%',
            colors=colors, startangle=90, wedgeprops={'edgecolor': 'white'})
axes[0].set_title('Distribución de Churn', fontsize=14, fontweight='bold')

# Barras
axes[1].bar(labels, churn_counts, color=colors, edgecolor='white')
axes[1].set_title('Cantidad de Clientes por Churn', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Cantidad de clientes')
for i, v in enumerate(churn_counts):
    axes[1].text(i, v + 30, str(v), ha='center', fontweight='bold')

plt.suptitle('Análisis General de Evasión de Clientes', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

tasa = df['Churn'].mean() * 100
print(f'📊 Tasa de evasión: {tasa:.1f}%')

In [ ]:
# ── Gráfico 2: Churn por variables demográficas ──
demograficas = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
titulos = ['Género', 'Adulto Mayor', 'Tiene Pareja', 'Tiene Dependientes']

fig, axes = plt.subplots(1, 4, figsize=(18, 5))

for ax, col, titulo in zip(axes, demograficas, titulos):
    churn_rate = df.groupby(col)['Churn'].mean() * 100
    bars = ax.bar(churn_rate.index, churn_rate.values,
                  color=['#3498db', '#e74c3c'], edgecolor='white')
    ax.set_title(titulo, fontsize=12, fontweight='bold')
    ax.set_ylabel('Tasa de Churn (%)')
    ax.set_ylim(0, 50)
    for bar in bars:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{bar.get_height():.1f}%', ha='center', fontweight='bold')

plt.suptitle('Tasa de Churn por Variables Demográficas', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ── Gráfico 3: Churn por tipo de contrato ──
fig, ax = plt.subplots(figsize=(10, 5))

churn_contrato = df.groupby('Contract')['Churn'].mean().sort_values(ascending=False) * 100
colores = ['#e74c3c', '#f39c12', '#2ecc71']

bars = ax.barh(churn_contrato.index, churn_contrato.values, color=colores, edgecolor='white')
ax.set_xlabel('Tasa de Churn (%)')
ax.set_title('Tasa de Churn por Tipo de Contrato', fontsize=14, fontweight='bold')
for bar in bars:
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f'{bar.get_width():.1f}%', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ── Gráfico 4: Churn por servicio de internet ──
fig, ax = plt.subplots(figsize=(8, 5))

churn_internet = df.groupby('InternetService')['Churn'].mean().sort_values(ascending=False) * 100

bars = ax.bar(churn_internet.index, churn_internet.values,
              color=['#e74c3c', '#f39c12', '#2ecc71'], edgecolor='white')
ax.set_ylabel('Tasa de Churn (%)')
ax.set_title('Tasa de Churn por Servicio de Internet', fontsize=14, fontweight='bold')
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            f'{bar.get_height():.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ── Gráfico 5: Distribución de tenure (meses) por Churn ──
fig, ax = plt.subplots(figsize=(10, 5))

for churn_val, label, color in [(0, 'No Churn', '#2ecc71'), (1, 'Churn', '#e74c3c')]:
    subset = df[df['Churn'] == churn_val]['tenure']
    ax.hist(subset, bins=30, alpha=0.6, label=label, color=color, edgecolor='white')

ax.set_xlabel('Meses de permanencia (tenure)')
ax.set_ylabel('Cantidad de clientes')
ax.set_title('Distribución de Permanencia por Churn', fontsize=14, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

print(df.groupby('Churn')['tenure'].describe())

In [ ]:
# ── Gráfico 6: Cargos mensuales por Churn ──
fig, ax = plt.subplots(figsize=(10, 5))

for churn_val, label, color in [(0, 'No Churn', '#2ecc71'), (1, 'Churn', '#e74c3c')]:
    subset = df[df['Churn'] == churn_val]['MonthlyCharges']
    ax.hist(subset, bins=30, alpha=0.6, label=label, color=color, edgecolor='white')

ax.set_xlabel('Cargo Mensual ($)')
ax.set_ylabel('Cantidad de clientes')
ax.set_title('Distribución de Cargos Mensuales por Churn', fontsize=14, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

print(df.groupby('Churn')['MonthlyCharges'].describe())

In [ ]:
# ── Gráfico 7: Método de pago vs Churn ──
fig, ax = plt.subplots(figsize=(12, 5))

churn_pago = df.groupby('PaymentMethod')['Churn'].mean().sort_values(ascending=False) * 100
colores = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']

bars = ax.barh(churn_pago.index, churn_pago.values, color=colores, edgecolor='white')
ax.set_xlabel('Tasa de Churn (%)')
ax.set_title('Tasa de Churn por Método de Pago', fontsize=14, fontweight='bold')
for bar in bars:
    ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2,
            f'{bar.get_width():.1f}%', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ── Gráfico 8: Heatmap de correlaciones ──
cols_numericas = ['Churn', 'tenure', 'MonthlyCharges', 'TotalCharges', 'DailyCharges']
corr = df[cols_numericas].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdYlGn',
            vmin=-1, vmax=1, ax=ax, linewidths=0.5)
ax.set_title('Mapa de Correlaciones', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ── Gráfico 9: Servicios adicionales vs Churn ──
servicios = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
             'TechSupport', 'StreamingTV', 'StreamingMovies']

churn_servicios = {}
for s in servicios:
    churn_servicios[s] = df[df[s] == 'Yes']['Churn'].mean() * 100

fig, ax = plt.subplots(figsize=(12, 5))
colors = ['#e74c3c' if v > 25 else '#3498db' for v in churn_servicios.values()]
bars = ax.bar(churn_servicios.keys(), churn_servicios.values(), color=colors, edgecolor='white')
ax.axhline(df['Churn'].mean() * 100, color='black', linestyle='--', label='Tasa global')
ax.set_ylabel('Tasa de Churn (%)')
ax.set_title('Tasa de Churn entre Clientes CON cada Servicio', fontsize=14, fontweight='bold')
ax.legend()
plt.xticks(rotation=15)
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
            f'{bar.get_height():.1f}%', ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

# 📄 Informe Final

## 🔍 Conclusiones del Análisis de Churn — TelecomX

---

### 1. Tasa de Evasión General
TelecomX presenta una tasa de churn de aproximadamente **26.5%**, lo que significa que 1 de cada 4 clientes abandona la empresa. Esta cifra es considerablemente alta para el sector de telecomunicaciones.

---

### 2. Factores con Mayor Impacto en el Churn

#### 📋 Tipo de Contrato — **Factor más determinante**
- Clientes con contrato **mes a mes** tienen una tasa de churn superior al **40%**.
- Clientes con contrato de **1 año** rondan el 11%.
- Clientes con contrato de **2 años** tienen menos del 3%.
- **Conclusión:** la falta de compromiso a largo plazo es el predictor más fuerte de abandono.

#### 🌐 Servicio de Internet
- Clientes con **Fibra Óptica** presentan la mayor tasa de churn (~42%).
- Esto puede indicar que el precio o la calidad del servicio de fibra no cumple las expectativas.

#### ⏳ Tiempo de permanencia (tenure)
- Los clientes que abandonan tienen una permanencia promedio de solo **~18 meses**, frente a **~37 meses** de los que se quedan.
- El **primer año** es el período más crítico de retención.

#### 💰 Cargos Mensuales
- Los clientes que hacen churn pagan en promedio **más por mes** (~$74 vs $61).
- Los clientes perciben que no reciben valor suficiente por el precio pagado.

#### 💳 Método de Pago
- El **cheque electrónico** tiene la tasa de churn más alta (~45%).
- Los pagos automáticos (tarjeta o débito) tienen tasas significativamente menores.

#### 👤 Variables Demográficas
- Los **adultos mayores** (SeniorCitizen) presentan una tasa de churn más alta (~41%).
- Clientes **sin pareja** ni **dependientes** tienen mayor propensión a abandonar.

---

### 3. Recomendaciones para el Equipo de Ciencia de Datos

1. **Priorizar** las variables `Contract`, `tenure`, `MonthlyCharges` e `InternetService` como predictores principales en el modelo.
2. **Segmentar** clientes de alto riesgo: mes a mes + fibra óptica + cheque electrónico.
3. Considerar campañas de **incentivos para migrar** a contratos anuales en los primeros 6 meses.
4. Revisar la **propuesta de valor del servicio de fibra óptica**.
5. Promover métodos de pago automático como mecanismo de retención.

---

*Análisis realizado como parte del desafío TelecomX — Alura LATAM*

In [ ]:
# Exportar el DataFrame limpio para el equipo de ciencia de datos
df.to_csv('TelecomX_clean.csv', index=False)
print('✅ Dataset limpio exportado como TelecomX_clean.csv')
print(f'Shape final: {df.shape}')
df.describe()